In [1]:
import warnings
warnings.filterwarnings('ignore', category = FutureWarning)
warnings.filterwarnings('ignore', category = DeprecationWarning)
warnings.filterwarnings('ignore', category = UserWarning)

In [2]:
import os
import re 
from ast import literal_eval
from random import seed,sample,choice
from itertools import chain
import multiprocessing
from nltk.tokenize import word_tokenize,WhitespaceTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from xgboost import XGBClassifier
import string
import stop_words
from string import punctuation
from nltk.corpus import stopwords
import nltk.corpus.util
from sklearn.tree import DecisionTreeClassifier
from nltk.classify import NaiveBayesClassifier
from keras.layers import Dense,Dropout
from keras.models import Sequential

Using TensorFlow backend.


In [253]:
cores = multiprocessing.cpu_count() - 1

In [112]:
df = pd.read_csv(r'.\Desktop\train_.csv',encoding = 'latin1')

In [87]:
df.columns

Index(['Unnamed: 0', 'sentiment', 'Sound Bite Text',
       'soundBites_with_hashtags'],
      dtype='object')

In [91]:
df.rename(columns = {'text':'Sound Bite Text'},inplace = True)

In [62]:
#Length of Dataframe
len(df)

4626

In [8]:
#No. of rows containing text data
len(df[df["Sound Bite Text"].isna() == False])

1887

In [9]:
#Rows not containing any text data
len(df[df["Sound Bite Text"].isna() == True])

0

In [10]:
df[df["Sound Bite Text"].isna() == True]

,sentiment,Sound Bite Text


# Data Cleaning

## Removing Hastags

In [89]:
#Rows not containing text data in "Sound Bite Text" column are dropped.
df.drop(df[df["Sound Bite Text"].isna() == True].index.values,axis = 0,inplace = True)

In [116]:
df = df.loc[df[df['Sentiments'].isna() == False].index.values]

KeyError: 'Sentiments'

In [12]:
#Creating seperate column for cleaned data(Without Hashtags)
df["soundBites"] = df["Sound Bite Text"].str.lower()

In [13]:
#Removing Hashtags
df["soundBites"] = df["soundBites"].apply(lambda row: re.sub(r'@[A-Z0-9a-z_]+', '',row))

In [14]:
df['soundBites'] = df["soundBites"].apply(lambda row: re.sub('[^a-zA-Z0-9#-_]',' ',row))

In [45]:
df['soundBites'] = df['soundBites'].apply(lambda row: re.sub('#\w+',' ',row))
df['soundBites'] = df['soundBites'].apply(lambda row: re.sub('[â❤]',' ',row))

In [46]:
#Removing Non-words
df['soundBites'] = df["soundBites"].apply(lambda row: re.sub('[;+>\/<,.)¿(&?:?%*=-]+','',row))
#df["soundBites"] = df["soundBites"].apply(lambda row: re.sub('\W+'," ",row))

In [47]:
#Removing URLs
df["soundBites"] = df["soundBites"].apply(lambda row: re.sub('http\S+','',row))

In [59]:
df['soundBites']

0                                       anything new could
1                                              little girl
2                  chicken noodles sold india halal please
3                                extreme affinity products
4                     follow follow us n promote us follow
5        tweet brasilia reuters brazilian agriculture m...
6        speaking congressional hearing tuesday said es...
7        pocket friendly cafe hangout folks serving uni...
8        beets pexelspixabay beets fiery deep color red...
9                                                         
10                                                        
11       reporting thomas mukoya george obulutsa duncan...
12       # hv never anything family biggest fan ur v ma...
13       place offers variety paranthas omelettes pasta...
14                                              april meri
15       noodles probably ultimate solution anytime kin...
16                                           yummy looki

In [49]:
#Removing Digits
df["soundBites"] = df["soundBites"].apply(lambda row: re.sub('\d+'," ",row))

In [50]:
df['soundBites'] = df["soundBites"].apply(lambda row: re.sub(r'\w+(.)$',' ',row))

In [51]:
df['soundBites'] = df["soundBites"].apply(lambda row: re.sub(r'\s+[.]',' ',row))

In [52]:
df['soundBites'] = df['soundBites'].apply(lambda row: row.split(' '))

In [53]:
#Filtering the empty strings from the list of tokens
df["soundBites"] = df["soundBites"].apply(lambda row: list(filter(None,row)))

In [54]:
#Creating list of stopwords
stopword = set(stopwords.words('english'))
stop_list = set(['maggi','maggie','also','nc','rz','njc','like','cz','gonna','carolina','garmin',
                 'screenshot','team','wish','provide','screen','rzd','moq','cgof','pcs','kis','ko','rt','customer','called',
                'informed','consumer'])
stopword.update(stop_list)

In [55]:
#Removing stopwords from tokens
df["soundBites"] = df["soundBites"].apply(lambda row :[item for item in row if item not in stopword])

In [56]:
df['soundBites']  = df['soundBites'].apply(' '.join)

In [58]:
wnl = WordNetLemmatizer()
df["soundBites"] = df["soundBites"].apply(lambda row: wnl.lemmatize(row))

## Cleaning (including hashtags)

In [113]:
#including Hashtags
df['soundBites_with_hashtags'] = df['Sound Bite Text'].str.lower()

In [114]:
#Removing words not containing characters,hashtags or underscores
df['soundBites_with_hashtags'] = df["soundBites_with_hashtags"].apply(lambda row: re.sub('[^a-zA-Z0-9#-_]',' ',row))

In [115]:
#Removing handles
df['soundBites_with_hashtags'] = df["soundBites_with_hashtags"].apply(lambda row: re.sub('@[a-zA-Z0-9_]+',' ',row))

In [116]:
#Removing Noise
df['soundBites_with_hashtags'] = df["soundBites_with_hashtags"].apply(lambda row: re.sub('[;+>.\/<,)❤☕₹⛩(&?:!%*=-]+','',row))

In [117]:
#Noise Removal
df['soundBites_with_hashtags'] = df["soundBites_with_hashtags"].apply(lambda row: re.sub('_{2,}','',row))

In [118]:
#Noise Removal
df["soundBites_with_hashtags"] = df["soundBites_with_hashtags"].apply(lambda row: re.sub('Â\x92',"\'",row))

In [119]:
#Noise Removal
df["soundBites_with_hashtags"] = df["soundBites_with_hashtags"].apply(lambda row: re.sub('Â\x91',"\'",row))

In [120]:
#Removing URLs
df["soundBites_with_hashtags"] = df["soundBites_with_hashtags"].apply(lambda row: re.sub('http\S+','',row))

In [121]:
df['soundBites_with_hashtags']

0       rt   'congrats to   mary sotos   for green pow...
1       pop up #bostonholiday  #urbannutcracker's   am...
2       new 4inch iphone 6 reported 5 reasons apple mu...
3         gbp200 repair an #iphone5 that died 4 mths o...
4       foreign currency exchange may be a headwind fo...
5       mark #zuckerberg slams tim #cook you don't car...
6           i can't say enough about how awesome apple...
7         #samsung #appeals $930m fine in   [#apple] #...
8                           block trade detected in #aapl
9       is apple inc losing the $5 billion edtech mark...
10      any one know how to retrieve deleted texts fro...
11      apple  fibonacci technicals levels  intraday  ...
12          #aaplapple is building a $161 million theatre
13      mark #zuckerberg slams tim #cook you're not in...
14      apple begins hiring for first belgian retail s...
15      apple launches ten holidaythemed itunes radio ...
16            'deleted #ipod #music from rival services' 
17      apple 

In [122]:
#Removing Full-Stops
df['soundBites_with_hashtags'] = df["soundBites_with_hashtags"].apply(lambda row: re.sub(r'[.]$',' ',row))

In [123]:
#removing dates and digits
df["soundBites_with_hashtags"] = df["soundBites_with_hashtags"].apply(lambda row: re.sub('\d+','',row))

In [124]:
#For Maggi Only
df['soundBites_with_hashtags'] = df['soundBites_with_hashtags'].apply(lambda row: row.replace('-Minute','2-Minute'))
df['soundBites_with_hashtags'] = df['soundBites_with_hashtags'].apply(lambda row: row.replace('-minute','2-Minute'))

In [125]:
# Seperating joined Hastags
df['soundBites_with_hashtags'] = df['soundBites_with_hashtags'].apply(lambda row: re.split(r'(#\w+)',row))

In [126]:
#Joining the terms in a document
df['soundBites_with_hashtags'] = df['soundBites_with_hashtags'].apply(' '.join)

In [127]:
#Lemmatization
wnl = WordNetLemmatizer()
df["soundBites_with_hashtags"] = df["soundBites_with_hashtags"].apply(lambda row: wnl.lemmatize(row))

In [128]:
#Splitting for filteration
df['soundBites_with_hashtags'] = df['soundBites_with_hashtags'].apply(lambda row: re.split(r' ',row))

In [129]:
#Filtering the empty strings from the list of tokens
df["soundBites_with_hashtags"] = df["soundBites_with_hashtags"].apply(lambda row: list(filter(None,row)))

In [130]:
#Removing Stopwords
stopword = set(stopwords.words('english'))
stopword_update = [item for item in stop_words.get_stop_words('en') if item not in nltk.corpus.stopwords.words('english')]
stop_list = set(['maggi','maggie','#maggi','#maggie','also','njc','nc','rz','use','like','carolina','food','love','garmin','new',
                 'raha','pm','tags','pak','across','ki','hi','screenshot','team','wish','provide','screen',
                 'pictwittercomclsauwtd',
                 '#foodie','#instafood','#foodblogger','tag','rt','updates','reuters','#fooddiaries','#foodisbae','#foodography',
                '#likeforlike','#followforfollow','follow','join','rs','#foodlove','delicious','#foodgasm','#foodielife','simpsons'
                 '#foodblogger','#merimaggie','#merimaggi','try','#indianblogger','#maggilove','#njcs','#maggielove','#foodporn',
                '#food','#foodie','#foodporn','#zomato','#maniamaggi','#tastymaggi','#maniamaggie','#tastymaggie','hai','u','km',
                 'tasty','yummy','#yummylicious','#yummilicious','#foddy','#foodgram','#maggilover','#maggielover','#tasty','rda',
                 '#yummy','#foddie','#swiggyindia','#zomatoindia','#swiggy','#foodstagram','#instagood','#indianfood','#likelike',
                 '#love','#delicious','#foodlover,''#picoftheday'',#noodles','#eeeats','#eeeeeats','#foodpics','#picoftheday',
                'noodles','#sodelhi','#likelike','#nomnom','#buzzfoodfeed','#fgram','#eatingforinsta','#saadidilli','kuch',
                '#saddidilli','#ll','#flt','#foodphotography','#foodtalkindia','#delhifoodie','#foodlover','#maggilovers',
                '#maggielovers','#foodtalkindia','#delhifoodblogger','#delhigram','#instagram','#instagood','#photooftheday',
                '#instadaily','#followfollow','#fgrams','#streetfood','#igers','#','#iger','#followme','#foodies','#likes',
                 '#instalike','#instalike','#indianfoodbloggers','indianfoodblogger','#foodshot','#foodcoma','gunsmoke','#noodles',
                '#vscodelhi','#foodblog','#foodiegram','#foodphoto','#delhi_igers','#delhifood','#delhidiaries','#mumbaifood',
                 '#mumbaidiaries','#foodoftheday','#indianfoodie','#foodaholic','#foods','#vscocam','#vscofood','#likeforfollow',
                '#delhifoodie','#streetfoods','#eat','#yum','#follow','th','us','#follow','#india','#ahmedabad','#chennai','#delhi',
                '#mumbai','#maggiloverforever','#vsco','#likesforlikes','#buzzfeedfood','#delhiblogger','#dfordelhi',
                '#tastyfood','#foodiesofinstagram','#knowyourmaggi'])
stopword.update(stop_list),
stopword.update(set(stopword_update))
stopword.update(set(string.ascii_lowercase))

In [131]:
#Removing stopwords from tokens
df["soundBites_with_hashtags"] = df["soundBites_with_hashtags"].apply(lambda row: [item for item in row if item not in stopword])

In [132]:
#Joining the terms in each Documents/
df["soundBites_with_hashtags"] = df["soundBites_with_hashtags"].apply(' '.join)

In [134]:
#Writing to File
df.to_csv(r'.\Desktop\train_.csv')

# Classification on training and test data

In [ ]:
data = pd.read_csv("Maggi_Clean2.csv")

In [137]:
#Extracting Classified data for training and test sets
data = df[df['sentiment'].isna() == False]

In [94]:
data.loc[data['Sentiments'] == 'None','Sentiments'] = 'Neutral'

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [133]:
#Creating seperate dataset for Classification
dataset = pd.DataFrame(data['soundBites_with_hashtags'])
dataset['Sentiments'] = data['sentiment']

In [169]:
#https://docs.python.org/2/library/ast.html#ast.literal_eval
#Used to convert a string containing a list into a list
#dataset['soundBites'] = dataset['soundBites'].apply(lambda row: literal_eval(''.join(row)))

In [ ]:
#Creating a string from list of tokens
dataset['soundBites'] = dataset['soundBites'].apply(' '.join)

In [44]:
dataset = df.copy()
dataset.rename(columns = {'Sentiment': 'Sentiments'},inplace = True)

In [108]:
#Splitting training and testing data
x_train,x_test,y_train,y_test = train_test_split(dataset['soundBites'],dataset['Sentiments'], random_state = 20, test_size = 0.3)

In [109]:
#Applying CountVectorizer to get token count
cv = CountVectorizer()
X_train = cv.fit_transform(x_train)
X_test = cv.transform(x_test)

In [111]:
#Applying TFIDF
tfidf = TfidfTransformer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

### Multinomial Naive Bayes

In [112]:
#Converting sparse training and testing datasets to dense matrices for classification
X_train = X_train.todense()
X_test = X_test.todense()
#Classification using Multinomial Naive Bayes
classifier = MultinomialNB()
classifier.fit(X_train,y_train)
prediction = classifier.predict(X_test)
accuracy_score(y_true = y_test, y_pred = prediction)

0.8898488120950324

### Gaussian Naive Bayes 

In [75]:
#Classifcation using Gaussian Naive Bayes
classifier = GaussianNB()
classifier.fit(X_train,y_train)
prediction = classifier.predict(X_test)
accuracy_score(y_true = y_test, y_pred = prediction)

0.5673146148308136

### Decision Tree Classifier

In [113]:
#Classification Using Decision Tree Classifier
clfdt = DecisionTreeClassifier(random_state = 42)
clfdt.fit(X_train,y_train)
prediction = clfdt.predict(X_test)
accuracy_score(y_true = y_test,y_pred = prediction)
print("Accuracy: ",accuracy_score(y_pred = prediction,y_true = y_test))
confusion_matrix(y_true = y_test, y_pred = prediction)

Accuracy:  0.9006479481641468


array([[ 17,  95,   5],
       [  9, 871,  19],
       [  1,   9, 363]], dtype=int64)

### Support Vector Classifier

In [77]:
#Classification Using Support Vector Classifier
clfSVM = LinearSVC()
clfSVM.fit(X_train,y_train)
prediction = clfSVM.predict(X_test)
print("Accuracy: ",accuracy_score(y_pred = prediction,y_true = y_test))
confusion_matrix(y_true = y_test, y_pred = prediction)

Accuracy:  0.912167026637869


array([[ 16,  97,   4],
       [  0, 890,   9],
       [  0,  12, 361]], dtype=int64)

### Random Forest Classifier

In [78]:
#Classification Using Random Forest Classifier
rfclf = RandomForestClassifier(random_state = 0)
rfclf.fit(X_train,y_train)
prediction = rfclf.predict(X_test)
accuracy_score(y_true = y_test, y_pred = prediction)
print("Accuracy: ",accuracy_score(y_pred = prediction,y_true = y_test))
confusion_matrix(y_true = y_test, y_pred = prediction)

Accuracy:  0.8984881209503239


array([[  9,  98,  10],
       [  2, 880,  17],
       [  0,  14, 359]], dtype=int64)

### XGBoost Classifier

In [60]:
#Classification using XGBoost Classifier
xgb = XGBClassifier(random_state = 21)
xgb.fit(X_train,y_train)
prediction = xgb.predict(X_test)
accuracy_score(y_true = y_test, y_pred = prediction)
print("Accuracy: ",accuracy_score(y_pred = prediction,y_true = y_test))
confusion_matrix(y_true = y_test, y_pred = prediction)

Accuracy:  0.9136069114470843


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([[ 20,  96,   1],
       [  0, 896,   3],
       [  4,  16, 353]], dtype=int64)

### Classification using Deeplearning - Keras

In [61]:
#Encoding the labels to be classified
labelEncoder = LabelEncoder()
Y_train = labelEncoder.fit_transform(y_train)
Y_test = labelEncoder.transform(y_test)

In [86]:
labelEncoder.classes_

array(['N', 'Neutral', 'P', 'P0'], dtype=object)

In [62]:
#One Hot Encoding the labels
one_hot_encoder = OneHotEncoder()
Y_train_ = one_hot_encoder.fit_transform(Y_train.reshape(-1,1))
Y_test_ = one_hot_encoder.transform(Y_test.reshape(-1,1))

In [64]:
#Deeplearning classification using Keras
seed(0)
model = Sequential()
model.add(Dense(output_dim =  32, input_shape = (X_train.shape[1],),activation = 'relu'))
model.add(Dense(output_dim = 32, activation = 'relu'))
model.add(Dense(output_dim = 32, activation = 'relu'))
model.add(Dense(output_dim = 32, activation = 'relu'))
model.add(Dense(output_dim = 4, activation = 'softmax'))
model.compile(optimizer = 'adagrad', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(X_train, Y_train_, batch_size = 100, epochs = 100)
y_pred = model.predict(X_test)

Epoch 1/100
3241/3241 [==============================] - 2s 563us/step - loss: 0.6463 - acc: 0.7877
Epoch 2/100
3241/3241 [==============================] - 0s 125us/step - loss: 0.2941 - acc: 0.9003
Epoch 3/100
3241/3241 [==============================] - 0s 130us/step - loss: 0.2481 - acc: 0.9087
Epoch 4/100
3241/3241 [==============================] - 0s 125us/step - loss: 0.2226 - acc: 0.9272
Epoch 5/100
3241/3241 [==============================] - 0s 130us/step - loss: 0.2074 - acc: 0.9324
Epoch 6/100
3241/3241 [==============================] - 0s 130us/step - loss: 0.1975 - acc: 0.9352
Epoch 7/100
3241/3241 [==============================] - 0s 125us/step - loss: 0.1922 - acc: 0.9358
Epoch 8/100
3241/3241 [==============================] - 0s 130us/step - loss: 0.1888 - acc: 0.9371
Epoch 9/100
3241/3241 [==============================] - 0s 125us/step - loss: 0.1874 - acc: 0.9367
Epoch 10/100
3241/3241 [==============================] - 0s 130us/step - loss: 0.1877 - acc: 0.9371

In [170]:
# Accuracy Score + Confusion Matrix
print('Accuracy = ',accuracy_score(y_true = Y_test, y_pred = y_pred.argmax(axis = 1)))
print(confusion_matrix(y_true = Y_test, y_pred = y_pred.argmax(axis = 1)))

NameError: name 'Y_test' is not defined

# Sentiment Analysis 
### using nltk.vader

In [65]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
data = pd.read_csv("Maggi_Clean.csv")

In [ ]:
# Assigning Sentiments based on Polarity Score
for item in data['Sound Bite Text']:   
    i = data.index.values[data['Sound Bite Text'] == item]
    scores = analyzer.polarity_scores(item)
    if (scores['compound'] < 0.05) & (scores['compound'] > -0.05):
        data.loc[i,'vader_Senti'] = 'None'
    elif scores['compound'] < -0.05:
        data.loc[i,'vader_Senti'] = 'N'
    elif scores['compound'] > 0.05:
        data.loc[i,'vader_Senti'] = 'P'

### Verification of Sentiments ( Manual Classification vs Vader)

In [ ]:
#Encoding labels 
le = LabelEncoder()
vader_in = le.fit_transform(data['Sentiment'])
vader_out = le.transform(data['vader_Senti'])

In [ ]:
accuracy_score(y_true = vader_out, y_pred  = vader_in)

In [ ]:
confusion_matrix(y_true = vader_out,y_pred = vader_in)

In [ ]:
data.loc[data['vader_Senti'] == 'N','Sound Bite Text'][42]

# Sentiment Classification
##### Training Data is manually classified
##### Entire dataset is classified as test data

In [138]:
# Creating Training Dataset
train = df.loc[df['sentiment'].dropna(axis = 0).index.values]

In [234]:
train = train.loc[train['soundBites_with_hashtags'].dropna(axis = 0).index.values]

In [139]:
X_train = train['soundBites_with_hashtags']
Y_train = train['sentiment']

In [140]:
#Creating test set
X_test = df['soundBites_with_hashtags']

In [141]:
cv = CountVectorizer()
X_train = cv.fit_transform(X_train)
X_test = cv.transform(X_test)

In [142]:
tfidf = TfidfTransformer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

In [143]:
#Converting sparse matrix to dense matrix
X_train = X_train.todense()
X_test = X_test.todense()

### Decision Tree Classifier

In [144]:
#DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train,Y_train)
df['Decision Tree'] = clf.predict(X_test)

### XGBoost Classifier

In [307]:
#Classification using XGBoost Classifier
xgb = XGBClassifier(random_state = 21,n_jobs = cores)
xgb.fit(X_train,Y_train)
df['XGB'] = xgb.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### Support Vector Classifier

In [308]:
#Classification Using Support Vector Classifier
clfSVM = LinearSVC()
clfSVM.fit(X_train,Y_train)
df['SVM'] = clfSVM.predict(X_test)

### Deeplearning - Keras

In [309]:
#Encoding the labels to be classified
labelEncoder = LabelEncoder()
Y_train = labelEncoder.fit_transform(Y_train)

In [278]:
labelEncoder.classes_

array(['N', 'Neutral', 'P'], dtype=object)

In [274]:
train.loc[train['Sentiments'] == 'p','Sentiments'] = 'P'

In [310]:
#One Hot Encoding the labels
one_hot_encoder = OneHotEncoder()
Y_train = one_hot_encoder.fit_transform(Y_train.reshape(-1,1))
#Y_test_ = one_hot_encoder.transform(Y_test.reshape(-1,1))

In [315]:
seed(0)
model = Sequential()
model.add(Dense(output_dim =  32, input_shape = (X_train.shape[1],),activation = 'relu'))
model.add(Dense(output_dim = 32, activation = 'relu'))
model.add(Dense(output_dim = 32, activation = 'relu'))
model.add(Dense(output_dim = 32, activation = 'relu'))
model.add(Dense(output_dim = 3, activation = 'softmax'))
model.compile(optimizer = 'adagrad', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(X_train, Y_train, batch_size = 100, epochs = 100)
y_pred = model.predict(X_test).argmax(axis = 1)

Epoch 1/100
736/736 [==============================] - 0s 488us/step - loss: 0.9048 - acc: 0.6712
Epoch 2/100
736/736 [==============================] - 0s 106us/step - loss: 0.5456 - acc: 0.7391
Epoch 3/100
736/736 [==============================] - 0s 127us/step - loss: 0.3765 - acc: 0.8030
Epoch 4/100
736/736 [==============================] - 0s 106us/step - loss: 0.2672 - acc: 0.9361
Epoch 5/100
736/736 [==============================] - 0s 106us/step - loss: 0.2029 - acc: 0.9497
Epoch 6/100
736/736 [==============================] - 0s 106us/step - loss: 0.1532 - acc: 0.9524
Epoch 7/100
736/736 [==============================] - 0s 106us/step - loss: 0.1228 - acc: 0.9538
Epoch 8/100
736/736 [==============================] - 0s 106us/step - loss: 0.1014 - acc: 0.9524
Epoch 9/100
736/736 [==============================] - 0s 106us/step - loss: 0.0806 - acc: 0.9633
Epoch 10/100
736/736 [==============================] - 0s 127us/step - loss: 0.0592 - acc: 0.9878
Epoch 11/100
736/73

In [324]:
y_pred = labelEncoder.inverse_transform(df['Keras_'])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [319]:
df['Keras'] = y_pred

In [145]:
df.to_csv(r'.\Desktop\train_.csv')